In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
np.random.seed(42)
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)


/Users/abhinav.sunderrajan/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Set hyperparameters

In [3]:
lr=0.1
epochs=10
n_output=10
n_input=784
n_dense1=64
n_dense2=64
batch_size=128
logs_path = "/tmp/mnist/2"
weights_init=tf.contrib.layers.xavier_initializer()

In [4]:
# A tensor flow place holder is usually something that is assigned at run time like the inputs and outs puts of a neural network
x=tf.placeholder(tf.float32,shape=[None,n_input])
y=tf.placeholder(tf.float32,shape=[None,n_output])

In [5]:
def dense(x,w,b):
    print(x.shape,w.shape,b.shape)
    z=tf.add(tf.matmul(x,w),b)
    a=tf.nn.relu(z)
    return a

In [6]:
def network(x,weights,biases):
    #two hidden layer
    dense_l=dense(x,weights['w1'],biases['b1'])
    dense_2=dense(dense_l,weights['w2'],biases['b2'])
    
    #output layer
    output=tf.add(tf.matmul(dense_2,weights['w_out']),biases['b_out'])
    return output
    

In [7]:
biases={'b1':tf.Variable(tf.zeros([n_dense1])),
        'b2':tf.Variable(tf.zeros([n_dense2])),
        'b_out':tf.Variable(tf.zeros([n_output]))
       }

In [8]:
weights={'w1': tf.get_variable('w1',shape=[n_input,n_dense1],initializer=weights_init),
         'w2': tf.get_variable('w2',shape=[n_dense1,n_dense2],initializer=weights_init),
         'w_out': tf.get_variable('w_out',shape=[n_dense2,n_output],initializer=weights_init) 
        }

### Define cost and optimizer

In [9]:
predictions=network(x,weights,biases)
# specify cost function
with tf.name_scope('cross_entropy'):
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions,labels=y))
    tf.summary.scalar("cross_entropy", cost)
# specify optimizer
with tf.name_scope('train'):    
    optimizer=tf.train.GradientDescentOptimizer(lr).minimize(cost)

(?, 784) (784, 64) (64,)
(?, 64) (64, 64) (64,)


### Define evaluation metrics

In [10]:
with tf.name_scope('Accuracy'):
    correct_prediction=tf.equal(tf.argmax(predictions,1),tf.argmax(y,1))
    acc_percent=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))*100.0

In [11]:
initializer_op=tf.global_variables_initializer()
# merge all summaries into a single "operation" which we can execute in a session 
summary_op = tf.summary.merge_all()

with tf.Session() as session:
    session.run(initializer_op)
    # create log writer object
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())   
    print("Training for",epochs," epochs")
    for epoch in range(epochs):
        n_batches=int(mnist.train.num_examples/batch_size)
        avg_cost=0.0
        avg_accuracy=0.0
        for batch in range(n_batches):
            batch_x,batch_y=mnist.train.next_batch(batch_size)
            _,summary,batch_cost,batch_acc=session.run([optimizer,summary_op,cost,acc_percent],feed_dict={x:batch_x,y:batch_y})
            # write log
            writer.add_summary(summary, epoch * n_batches + batch)
            
            avg_cost+=batch_cost/n_batches
            avg_accuracy+=batch_acc/n_batches
        print("epoch:",epoch,"cost:",avg_cost,"accuracy:",avg_accuracy)

    print("Training cost:",cost.eval({x:mnist.test.images,y:mnist.test.labels}))
    print("Test accuracy:",acc_percent.eval({x:mnist.test.images,y:mnist.test.labels}))
            
        

Training for 10  epochs
epoch: 0 cost: 0.47858566542466513 accuracy: 86.31446678321677
epoch: 1 cost: 0.23316689448687306 accuracy: 93.2200611888113
epoch: 2 cost: 0.17773384395943373 accuracy: 94.82990967365969
epoch: 3 cost: 0.14582261101001906 accuracy: 95.75320512820517
epoch: 4 cost: 0.12342253788037864 accuracy: 96.45979020979023
epoch: 5 cost: 0.10800330730324434 accuracy: 96.84404137529138
epoch: 6 cost: 0.09543500091264025 accuracy: 97.18822843822852
epoch: 7 cost: 0.08518031820880644 accuracy: 97.51602564102559
epoch: 8 cost: 0.0772832899651814 accuracy: 97.76187354312357
epoch: 9 cost: 0.07054456530056355 accuracy: 97.89481351981362
Training cost: 0.09072857
Test accuracy: 96.93
